# Data Engineering Capstone Project - Metro_Madrid

## Project Summary
This notebook gets output data from [metro-big-data-unir](https://github.com/juananthony/metro-big-data-unir) project and create a database to analysis.

[*Metro de Madrid*](https://www.metromadrid.es/) is the name of the tube/subway service that operates in Madrid, Spain. This service has 302 stations on 13 lines plus a light rail system called *Metro Ligero*. This service is used, on average in 2020, more than 30 million times each month.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
import os
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('udacity-capstone').getOrCreate()
spark.conf.set('spark.sql.session.timeZone', 'CET')

## Step 1: Scope the Project and Gather Data

### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [36]:
DATA_PATH = './data'
lines_file = 'lines.csv'
stations_file = 'stations.csv'
mentions_file = 'mentions_20210126.csv'

In [37]:
lines = spark.read.csv(os.path.join(DATA_PATH, lines_file), header=True)
lines.count()

16

In [38]:
stations = spark.read.csv(os.path.join(DATA_PATH, stations_file), header=True)
stations.count()

332

In [39]:
mentions = spark.read.csv(os.path.join(DATA_PATH, mentions_file), header=True, multiLine=True, escape='"')
mentions.count()

946929

In [40]:
mentions.printSchema()

root
 |-- _id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- text: string (nullable = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- user.id: string (nullable = true)
 |-- user.name: string (nullable = true)
 |-- user.screen_name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- profile_image_url: string (nullable = true)
 |-- profile_image_url_https: string (nullable = true)
 |-- extended_tweet.full_text: string (nullable = true)
 |-- classification.naive_bayes.result: string (nullable = true)



In [41]:
mentions.groupBy('`classification.naive_bayes.result`').count().show()

+---------------------------------+------+
|classification.naive_bayes.result| count|
+---------------------------------+------+
|                             null| 83101|
|                          nothing|693429|
|                        complaint|111098|
|                            issue| 59301|
+---------------------------------+------+



## Step 2: Explore and Assess the Data
### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

### Cleaning Steps
Document steps necessary to clean the data

In [42]:
from pyspark.sql.functions import col, to_timestamp
classes = ['complaint', 'issue']
mentions = mentions.filter(col('`classification.naive_bayes.result`').isin(classes))

In [43]:
mentions = mentions.withColumn('dt', to_timestamp(mentions.created_at, 'E MMM d HH:m:s Z y').alias('dt'))

In [59]:
from pyspark.sql.functions import when
mentions = mentions.withColumn('full_text', 
                               when(~col('`extended_tweet.full_text`').isNull(), col('`extended_tweet.full_text`'))
                               .otherwise(col('text'))).drop('text','`extended_tweet.full_text`')

## Step 3: Define the Data Model
### 3.1 Conceptual Data Model
The data we want to store is all messages that inform about any issue or complaint in a line or a station even if one message inform about an issue that affect two different lines. That the reason why the fact table is the inform fact, that can be a complaint or an issue. One tweet can inform about an issue that affect two lines (i.e.: a closed station and all lines that stops there). In other words, one tweet generates one or many "inform fact" records.

![fact-dimension diagram](./img/class_diagram.png "Fact-Dimension Diagram")

### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model
 
#### Line Dimension

In [10]:
from pyspark.sql.functions import monotonically_increasing_id
lines_dim = lines.withColumn('line_id', monotonically_increasing_id()).withColumnRenamed('line','line_name')
lines_dim.show(5)

+---------+-------+
|line_name|line_id|
+---------+-------+
|       L9|      0|
|      ML3|      1|
|       L8|      2|
|      ML1|      3|
|      ML2|      4|
+---------+-------+
only showing top 5 rows



#### Station Dimension

In [11]:
stations_dim = stations.join(lines_dim, lines_dim.line_name == stations.line, 'inner').select(monotonically_increasing_id().alias('station_id'), stations.station.alias('station_name'), lines_dim.line_id)
stations_dim.show(5)

+----------+----------------+-------+
|station_id|    station_name|line_id|
+----------+----------------+-------+
|         0|   Paco de Lucía|      0|
|         1|      Mirasierra|      0|
|         2|    Herrera Oria|      0|
|         3|Barrio del Pilar|      0|
|         4|        Ventilla|      0|
+----------+----------------+-------+
only showing top 5 rows



#### Class Diagram

In [12]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number

class_df = mentions.select(col('`classification.naive_bayes.result`').alias('class_name')).distinct()
w = Window.orderBy('class_name')

class_df.withColumn('class_id', row_number().over(w)).show()

+----------+--------+
|class_name|class_id|
+----------+--------+
| complaint|       1|
|     issue|       2|
+----------+--------+



#### Mapping date dimensions

In [13]:
from pyspark.sql.functions import year, month, quarter, dayofweek, dayofmonth, hour, minute

dates_df = mentions.select(
                        year(col('dt')).alias('year'),
                        quarter(col('dt')).alias('quarter'),
                        month(col('dt')).alias('month'),
                        dayofmonth(col('dt')).alias('day'),
                        dayofweek(col('dt')).alias('weekday'),
                        hour(col('dt')).alias('hour'),
                        minute(col('dt')).alias('minute')) \
                .dropDuplicates() \
                .withColumn('date_id', monotonically_increasing_id())

dates_df.show(10, False)

+----+-------+-----+---+-------+----+------+-------+
|year|quarter|month|day|weekday|hour|minute|date_id|
+----+-------+-----+---+-------+----+------+-------+
|2018|4      |11   |17 |7      |19  |42    |0      |
|2018|4      |11   |20 |3      |11  |56    |1      |
|2018|4      |11   |26 |2      |12  |22    |2      |
|2018|4      |11   |30 |6      |0   |24    |3      |
|2018|4      |11   |30 |6      |14  |35    |4      |
|2018|4      |11   |30 |6      |19  |47    |5      |
|2018|4      |11   |30 |6      |20  |44    |6      |
|2018|4      |12   |1  |7      |9   |3     |7      |
|2018|4      |12   |6  |5      |9   |50    |8      |
|2018|4      |12   |6  |5      |10  |56    |9      |
+----+-------+-----+---+-------+----+------+-------+
only showing top 10 rows



#### Tweet Dimension

In [60]:
from pyspark.sql.functions import substring

mentions.join(dates_df,
              (year(mentions.dt) == dates_df.year) &
              (month(mentions.dt) == dates_df.month) &
              (dayofmonth(mentions.dt) == dates_df.day) &
              (hour(mentions.dt) == dates_df.hour) &
              (minute(mentions.dt) == dates_df.minute),
              'inner'
             ) \
        .select(col('id').alias('tweet_id'),
                'date_id',
                col('`user.id`').alias('user_id'),
                col('full_text').alias('text'),
                col('in_reply_to_status_id').alias('reply_tweet_id')
               ).show(1,False)

+-------------------+------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+
|tweet_id           |date_id     |user_id  |text                                                                                                                                                          |reply_tweet_id|
+-------------------+------------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+
|1062598007793860609|292057776270|289007361|@metro_madrid y ¿hoy que le pasa a la Línea 10 dirección Tres Olivos? ¿Por qué nos paramos en todas las estaciones? Todos los días una aventura! #máximapereza|null          |
+-------------------+------------+---------+--------------------------------------------------------------------------------

In [27]:
mentions.select('_id', substring(col('_id'), 10, 24)).show(20, False)

+----------------------------------+------------------------+
|_id                               |substring(_id, 10, 24)  |
+----------------------------------+------------------------+
|ObjectId(5be9f6d0fab7400004ae89c8)|5be9f6d0fab7400004ae89c8|
|ObjectId(5be9ee7520a224000468a64a)|5be9ee7520a224000468a64a|
|ObjectId(5bea7501fab7400004ae89e0)|5bea7501fab7400004ae89e0|
|ObjectId(5bea6dd0fab7400004ae89d4)|5bea6dd0fab7400004ae89d4|
|ObjectId(5bea75c5fab7400004ae89e6)|5bea75c5fab7400004ae89e6|
|ObjectId(5bea08b5212c7f0004857734)|5bea08b5212c7f0004857734|
|ObjectId(5bea7258fab7400004ae89da)|5bea7258fab7400004ae89da|
|ObjectId(5bea1256212c7f000485799f)|5bea1256212c7f000485799f|
|ObjectId(5bea1614212c7f0004857aaa)|5bea1614212c7f0004857aaa|
|ObjectId(5bea7a59fab7400004ae8a00)|5bea7a59fab7400004ae8a00|
|ObjectId(5bea139a212c7f00048579eb)|5bea139a212c7f00048579eb|
|ObjectId(5bea766bfab7400004ae89ec)|5bea766bfab7400004ae89ec|
|ObjectId(5beaad42212c7f00048599d4)|5beaad42212c7f00048599d4|
|ObjectI

## Step 4: Run Pipelines to Model the Data 
### 4.1 Create the data model
Build the data pipelines to create the data model.

### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.